# 200520
OpenPose

1. OpenCV 설치
    >pip install opencv-python
2. MPI 및 COCO 모델 다운로드 후 model/로 이동, coco.caffemodel  coco.prototxt  mpi.caffemodel  mpi.prototxt  test.md
##### MPII Model(caffe/prototxt)
    http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/mpi/pose_iter_160000.caffemodel
    
    이름 변경!! pose_iter_160000.caffemodel -> mpi.caffemodel
    http://blogattach.naver.net/bf2aa3196d5a318ea4442f1a22c4b5c163344f/20200220_77_blogfile/shino1025_1582181201869_SH72yM_prototxt/mpi.prototxt
##### COCO Model(caffe/prototxt)
    http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/coco/pose_iter_440000.caffemodel

    이름 변경!! pose_iter_440000.caffemodel -> coco.caffemodel
   http://blogattach.naver.net/ac39b00016272690b657360d34d7addf7222d9dc/20200220_183_blogfile/shino1025_1582181217717_FTsb78_prototxt/coco.prototxt
    

3. 이미지/영상 탐지 - output/에 결과 생성
    > python image.py

    > python video.py
    
    
    
> 200MB 모델 수행시 1~1.5초 소요, CPU가 더 빠르다

In [1]:
##
import cv2
import time
import numpy as np
import sys
import os

USE_GPU = False # False

def proc(module, img):
    if module == "mpi":
        protoFile   = "./model/mpi.prototxt"
        weightsFile = "./model/mpi.caffemodel"
        nPoints = 15  # data에 있는 point 갯수
        POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]
    else:
        protoFile   = "./model/coco.prototxt"
        weightsFile = "./model/coco.caffemodel"
        nPoints = 18
        POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
    print("input/" + img)
    frame = cv2.imread(os.path.join('input', img))
    frameCopy   = np.copy(frame)
    frameWidth  = frame.shape[1]
    frameHeight = frame.shape[0]
    print('(H,W,C) :',(frame.shape))
    threshold = 0.1
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    
    # check if we are going to use GPU
    if USE_GPU:
        print('USE GPU!!')
        # set CUDA as the preferable backend and target
        print("[INFO] setting preferable backend and target to CUDA...")
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
    
    
    start_time = time.time()  # 수행시간 측정
    inWidth  = 368
    inHeight = 368

    # inpBlob : (425, 650, 3)->(1, 3, W, H), 0~255->0~1로 normalize, (W, H)로 resize
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)
    net.setInput(inpBlob)
    output = net.forward()  # (N, 57, 46, 46)

    print("%s - 추론 완료 : %.1f sec"%(module, time.time() - start_time))    
    
    H = output.shape[2]
    W = output.shape[3]
    points = []
    ## 각 좌표들 찍기
    for i in range(nPoints):    
        probMap = output[0, i, :, :]  # (46, 46), batch에 여러개가 있을 시, 이부분 수정 0->?
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H
        # 최대값이 thr보다 높을 때 그 좌표로 간주
        if prob > threshold :
            # 가장 높은 위치에 원 그리기
            frameCopy = cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            # txt 부치기
            frameCopy = cv2.putText(frameCopy, 'OpenCV', (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else :
            points.append(None)  # 없으면 없구나 하고 pass    
    ## 관절과 관절 사이 선 그리기
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        if points[partA] and points[partB]:
            frameCopy = cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
            frameCopy = cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
    cv2.imwrite(os.path.join('output', module + "_" + img), frameCopy)  # 이미지파일 저장

if __name__ == '__main__':
    module = ["mpi","coco"]
    for i in module:
        for j in os.listdir('input'):
            if j[-3:] in ['jpg','png','peg']:
                proc(i, j)
                
    print('done!')

input/tennis1.jpg
(H,W,C) : (425, 650, 3)
mpi - 추론 완료 : 1.1 sec
input/tennis2.jpg
(H,W,C) : (356, 500, 3)
mpi - 추론 완료 : 1.0 sec
input/tennis1.jpg
(H,W,C) : (425, 650, 3)
coco - 추론 완료 : 1.5 sec
input/tennis2.jpg
(H,W,C) : (356, 500, 3)
coco - 추론 완료 : 1.5 sec
